In [1]:
import sys 
sys.path.insert(0, '/home/yikuan/project/Code/')

In [2]:
from common.pytorch import save_model
from common.common import create_folder, H5Recorder, load_obj, H5Recorder
import math
import os
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from ACM.model.utils.utils import age_vocab
import pandas as pd
import pytorch_pretrained_bert as Bert
from ACM.dataLoader.HF import HF_data
from ACM.model.bertWhitenedGP import BertHF as BertHF_GPAbs
from ACM.model.bertKISSGP import BertHF as BertHF_GPAdditive
from ACM.model.behrt import BertHF as BertHF_BERT
from ACM.model.bertDBKLEmbedding import BertHF as BertHF_BNNGP
from ACM.model.bertBayesianEmbedding import BertHF as BertHF_BNN
from ACM.model.bertBayesianOutput import BertHF as BertHF_BNNClassifier
from ACM.model.bertBayesianEmbeddingOutput import BertHF as BertHF_BNNEmbClassifier
import gpytorch
from ACM.model.optimiser import adam
import torch.nn as nn
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
# from Utils.evaluation import ECELoss,uncertainty_divergence
from matplotlib.pyplot import hist
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ACM.evaluation.metrics import uncertain_cal
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from sklearn.calibration import calibration_curve

In [3]:
# Model GPAbs
class BertConfigGPAbs(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfigGPAbs, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings=config.get('max_position_embeddings'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        self.num_labels = config.get('num_labels')

class TrainConfigGPAbs(object):
    def __init__(self, config):
        self.batch_size = config.get('batch_size')
        self.use_cuda = config.get('use_cuda')
        self.max_len_seq = config.get('max_len_seq')
        self.train_loader_workers = config.get('train_loader_workers')
        self.test_loader_workers = config.get('test_loader_workers')
        self.device = config.get('device')
        self.output_dir = config.get('output_dir')
        self.output_name = config.get('output_name')
        self.best_name = config.get('best_name')
        
# Model GPAdditive
class BertConfigGPAdditive(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfigGPAdditive, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings=config.get('max_position_embeddings'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        self.num_labels = config.get('num_labels')
        
class BertConfigBNN(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfigBNN, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings=config.get('max_position_embeddings'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        self.num_labels = config.get('num_labels')

class BertConfigBNNGP(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfigBNNGP, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings=config.get('max_position_embeddings'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        self.num_labels = config.get('num_labels')
        self.prior_rate = config.get('prior_rate')
        
class BertConfigBNNClassifier(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfigBNNClassifier, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings=config.get('max_position_embeddings'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        self.num_labels = config.get('num_labels')
        
# Model Bert
class BertConfigBERT(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfig, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings=config.get('max_position_embeddings'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        self.num_labels = config.get('num_labels')
        
class BertConfigEmbClassifier(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfigEmbClassifier, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings=config.get('max_position_embeddings'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        self.num_labels = config.get('num_labels')

class TrainConfigGPAdditive(object):
    def __init__(self, config):
        self.batch_size = config.get('batch_size')
        self.use_cuda = config.get('use_cuda')
        self.max_len_seq = config.get('max_len_seq')
        self.train_loader_workers = config.get('train_loader_workers')
        self.test_loader_workers = config.get('test_loader_workers')
        self.device = config.get('device')
        self.output_dir = config.get('output_dir')
        self.output_name = config.get('output_name')
        self.best_name = config.get('best_name')

class TrainConfigBERT(object):
    def __init__(self, config):
        self.batch_size = config.get('batch_size')
        self.use_cuda = config.get('use_cuda')
        self.max_len_seq = config.get('max_len_seq')
        self.train_loader_workers = config.get('train_loader_workers')
        self.test_loader_workers = config.get('test_loader_workers')
        self.device = config.get('device')
        self.output_dir = config.get('output_dir')
        self.output_name = config.get('output_name')
        self.best_name = config.get('best_name')

class TrainConfigBNNGP(object):
    def __init__(self, config):
        self.batch_size = config.get('batch_size')
        self.use_cuda = config.get('use_cuda')
        self.max_len_seq = config.get('max_len_seq')
        self.train_loader_workers = config.get('train_loader_workers')
        self.test_loader_workers = config.get('test_loader_workers')
        self.device = config.get('device')
        self.output_dir = config.get('output_dir')
        self.output_name = config.get('output_name')
        self.best_name = config.get('best_name')

class TrainConfigBNN(object):
    def __init__(self, config):
        self.batch_size = config.get('batch_size')
        self.use_cuda = config.get('use_cuda')
        self.max_len_seq = config.get('max_len_seq')
        self.train_loader_workers = config.get('train_loader_workers')
        self.test_loader_workers = config.get('test_loader_workers')
        self.device = config.get('device')
        self.output_dir = config.get('output_dir')
        self.output_name = config.get('output_name')
        self.best_name = config.get('best_name')
        self.num_samples = config.get('num_samples')

class TrainConfigBNNClassifier(object):
    def __init__(self, config):
        self.batch_size = config.get('batch_size')
        self.use_cuda = config.get('use_cuda')
        self.max_len_seq = config.get('max_len_seq')
        self.train_loader_workers = config.get('train_loader_workers')
        self.test_loader_workers = config.get('test_loader_workers')
        self.device = config.get('device')
        self.output_dir = config.get('output_dir')
        self.output_name = config.get('output_name')
        self.best_name = config.get('best_name')
        self.num_samples = config.get('num_samples')

class TrainConfigEmbClassifier(object):
    def __init__(self, config):
        self.batch_size = config.get('batch_size')
        self.use_cuda = config.get('use_cuda')
        self.max_len_seq = config.get('max_len_seq')
        self.train_loader_workers = config.get('train_loader_workers')
        self.test_loader_workers = config.get('test_loader_workers')
        self.device = config.get('device')
        self.output_dir = config.get('output_dir')
        self.output_name = config.get('output_name')
        self.best_name = config.get('best_name')
        self.num_samples = config.get('num_samples')

In [4]:
file_config = {
    'vocab':'/home/shared/yikuan/HF/Data/PureICD/Full_vocab',
    'train': '/home/shared/01_data/03_cuts/01_model_improvement/separate_med_diag/trainV1BertDM.parquet',
    'test': '/home/shared/01_data/03_cuts/01_model_improvement/separate_med_diag/testV1BertDM.parquet'
}

In [5]:
global_params = {
    'max_seq_len': 256,
    'max_age': 110,
    'month': 1,
    'age_symbol': None,
}

optim_param = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

In [6]:
BertVocab = load_obj(file_config['vocab'])
ageVocab, idx2age = age_vocab(max_age=global_params['max_age'], mon=global_params['month'], symbol=global_params['age_symbol'])

In [7]:
trainData = pd.read_parquet(file_config['train']).reset_index(drop=True)
testData = pd.read_parquet(file_config['test']).reset_index(drop=True)

In [8]:
train_params_GPAbs = {
    'batch_size': 64,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'train_loader_workers': 3,
    'test_loader_workers': 0
}

train_params_GPAdditive = {
    'batch_size': 64,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'train_loader_workers': 3,
    'test_loader_workers': 3
}

train_params_BERT = {
    'batch_size': 64,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'train_loader_workers': 3,
    'test_loader_workers': 3
}

train_params_BNNGP = {
    'batch_size': 64,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'train_loader_workers': 3,
    'test_loader_workers': 3
}

train_params_BNN = {
    'batch_size': 64,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'train_loader_workers': 3,
    'test_loader_workers': 3,
    'num_samples': 10
}

train_params_BNNClassifier = {
    'batch_size': 64,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'train_loader_workers': 3,
    'test_loader_workers': 3,
    'num_samples': 10
}

train_params_EmbClassifier = {
    'batch_size': 64,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'train_loader_workers': 3,
    'test_loader_workers': 3,
    'num_samples': 10
}

In [10]:
trainConfig_GPAbs = TrainConfigGPAbs(train_params_GPAbs)
trainConfig_GPAdditive = TrainConfigGPAdditive(train_params_GPAdditive)
trainConfig_BERT = TrainConfigBERT(train_params_BERT)
trainConfig_BNN = TrainConfigBNN(train_params_BNN)
trainConfig_BNNGP = TrainConfigBNNGP(train_params_BNNGP)
trainConfig_BNNClassifier = TrainConfigBNNGP(train_params_BNNClassifier)
trainConfig_EmbClassifier = TrainConfigEmbClassifier(train_params_EmbClassifier)

data_set = HF_data(trainConfig_GPAbs, trainData, testData, BertVocab['token2idx'], ageVocab, code='code', age='age')

trainData = data_set.get_weighted_sample_train(4)
testData = data_set.get_test_loader()

In [11]:
model_param_GPAbs = {
    'vocab_size': len(BertVocab['token2idx'].keys()),
    'hidden_size': 150,
    'num_hidden_layers': 4,
    'num_attention_heads': 6,
    'hidden_act': 'gelu',
    'intermediate_size': 108,
    'max_position_embeddings': global_params['max_seq_len'],
    'seg_vocab_size': 2,
    'age_vocab_size': len(ageVocab.keys()),
    'prior_prec': 1e0,
    'prec_init': 1e0,
    'initializer_range': 0.02,
    'num_labels': 1,
    'hidden_dropout_prob': 0.29,
    'attention_probs_dropout_prob': 0.38
}

model_param_GPAdditive = {
    'vocab_size': len(BertVocab['token2idx'].keys()),
    'hidden_size': 150,
    'num_hidden_layers': 4,
    'num_attention_heads': 6,
    'hidden_act': 'gelu',
    'intermediate_size': 108,
    'max_position_embeddings': global_params['max_seq_len'],
    'seg_vocab_size': 2,
    'age_vocab_size': len(ageVocab.keys()),
    'prior_prec': 1e0,
    'prec_init': 1e0,
    'initializer_range': 0.02,
    'num_labels': 1,
    'hidden_dropout_prob': 0.29,
    'attention_probs_dropout_prob': 0.38
}

model_param_BERT = {
    'vocab_size': len(BertVocab['token2idx'].keys()),
    'hidden_size': 150,
    'num_hidden_layers': 4,
    'num_attention_heads': 6,
    'hidden_act': 'gelu',
    'intermediate_size': 108,
    'max_position_embeddings': global_params['max_seq_len'],
    'seg_vocab_size': 2,
    'age_vocab_size': len(ageVocab.keys()),
    'prior_prec': 1e0,
    'prec_init': 1e0,
    'initializer_range': 0.02,
    'num_labels': 1,
    'hidden_dropout_prob': 0.29,
    'attention_probs_dropout_prob': 0.38
}

model_param_BNNGP = {
    'vocab_size': len(BertVocab['token2idx'].keys()),
    'hidden_size': 150,
    'num_hidden_layers': 4,
    'num_attention_heads': 6,
    'hidden_act': 'gelu',
    'intermediate_size': 108,
    'max_position_embeddings': global_params['max_seq_len'],
    'seg_vocab_size': 2,
    'age_vocab_size': len(ageVocab.keys()),
    'prior_prec': 1e0,
    'prec_init': 1e0,
    'initializer_range': 0.02,
    'num_labels': 1,
    'hidden_dropout_prob': 0.29,
    'attention_probs_dropout_prob': 0.38,
    'prior_rate': 0.347
}

model_param_BNN = {
    'vocab_size': len(BertVocab['token2idx'].keys()),
    'hidden_size': 150,
    'num_hidden_layers': 4,
    'num_attention_heads': 6,
    'hidden_act': 'gelu',
    'intermediate_size': 108,
    'max_position_embeddings': global_params['max_seq_len'],
    'seg_vocab_size': 2,
    'age_vocab_size': len(ageVocab.keys()),
    'prior_prec': 1e0,
    'prec_init': 1e0,
    'initializer_range': 0.02,
    'num_labels': 1,
    'hidden_dropout_prob': 0.29,
    'attention_probs_dropout_prob': 0.38
}

model_param_BNNClassifier = {
    'vocab_size': len(BertVocab['token2idx'].keys()),
    'hidden_size': 150,
    'num_hidden_layers': 4,
    'num_attention_heads': 6,
    'hidden_act': 'gelu',
    'intermediate_size': 108,
    'max_position_embeddings': global_params['max_seq_len'],
    'seg_vocab_size': 2,
    'age_vocab_size': len(ageVocab.keys()),
    'prior_prec': 1e0,
    'prec_init': 1e0,
    'initializer_range': 0.02,
    'num_labels': 1,
    'hidden_dropout_prob': 0.29,
    'attention_probs_dropout_prob': 0.38
}

model_param_EmbClassifier = {
    'vocab_size': len(BertVocab['token2idx'].keys()),
    'hidden_size': 150,
    'num_hidden_layers': 4,
    'num_attention_heads': 6,
    'hidden_act': 'gelu',
    'intermediate_size': 108,
    'max_position_embeddings': global_params['max_seq_len'],
    'seg_vocab_size': 2,
    'age_vocab_size': len(ageVocab.keys()),
    'prior_prec': 1e0,
    'prec_init': 1e0,
    'initializer_range': 0.02,
    'num_labels': 1,
    'hidden_dropout_prob': 0.29,
    'attention_probs_dropout_prob': 0.38
}

feature_dict = {
    'word': True,
    'age': False,
    'seg': False,
    'norm': True
}

In [12]:
modelConfigGPAbs = BertConfigGPAbs(model_param_GPAbs)
modelConfigGPAdditive = BertConfigGPAdditive(model_param_GPAdditive)
modelConfigBERT = BertConfigGPAdditive(model_param_BERT)
modelConfigBNN = BertConfigBNN(model_param_BNN)
modelConfigBNNGP = BertConfigBNNGP(model_param_BNNGP)
modelConfigBNNClassifier = BertConfigBNNClassifier(model_param_BNNClassifier)
modelConfigEmbClassifier = BertConfigEmbClassifier(model_param_EmbClassifier)

In [13]:
# Model GPAbs
cuda_gpAbs = 'cuda:1'
model_GPAbs = BertHF_GPAbs(modelConfigGPAbs, n_dim=20,grid_size=40, ard_num_dims=20)
likelihoodAbs = gpytorch.likelihoods.BernoulliLikelihood().to(cuda_gpAbs)
mllAbs = gpytorch.mlls.VariationalELBO(likelihoodAbs, model_GPAbs.gp_layer, num_data=len(trainData.dataset))

# Model GPAdditive
cuda_gpAdditive = 'cuda:1'
model_GPAdditive = BertHF_GPAdditive(modelConfigGPAdditive, n_dim=24,grid_size=40, ard_num_dims=24)
likelihoodAdditive = gpytorch.likelihoods.BernoulliLikelihood().to(cuda_gpAdditive)
mllAdditive = gpytorch.mlls.VariationalELBO(likelihoodAdditive, model_GPAdditive.gp_layer, num_data=len(trainData.dataset))

# Model BERT
cuda_bert = 'cuda:1'
model_BERT = BertHF_BERT(modelConfigBERT, num_labels=modelConfigBERT.num_labels, n_dim=150)

# BNNGP
# cuda_bnngp_0 = 'cuda:0'
# cuda_bnngp_1 = 'cuda:1'
# model_BNNGP = BertHF_BNNGP(modelConfigBNNGP, n_dim=24,grid_size=40, ard_num_dims=24, feature_dict=feature_dict)
# likelihoodBNNGP = gpytorch.likelihoods.BernoulliLikelihood().to(cuda_bnngp)
# mllBNNGP = gpytorch.mlls.VariationalELBO(likelihoodBNNGP, model_BNNGP.gp_layer, num_data=len(trainData.dataset))
cuda_bnngp_0 = 'cuda:1'
cuda_bnngp_1 = 'cuda:1'
model_BNNGP = BertHF_BNNGP(modelConfigBNNGP, n_dim=24,grid_size=40, ard_num_dims=24, feature_dict=feature_dict, cuda1=cuda_bnngp_0, cuda2=cuda_bnngp_1)
likelihoodBNNGP = gpytorch.likelihoods.BernoulliLikelihood().to(cuda_bnngp_1)
mllBNNGP = gpytorch.mlls.VariationalELBO(likelihoodBNNGP, model_BNNGP.gp_layer, num_data=len(trainData.dataset))

# BNN
cuda_bnn = 'cuda:1'
model_BNN = BertHF_BNN(modelConfigBNN, num_labels=modelConfigBNN.num_labels, n_dim=150, feature_dict=feature_dict)

# BNNClassifier
cuda_bnnclassifier = 'cuda:1'
model_BNNClassifier = BertHF_BNNClassifier(modelConfigBNNClassifier, num_labels=modelConfigBNNClassifier.num_labels, n_dim=150)

# EmbClassifier
cuda_embclassifier = 'cuda:1'
model_EmbClassifier = BertHF_BNNEmbClassifier(modelConfigEmbClassifier, num_labels=modelConfigEmbClassifier.num_labels, n_dim=150, feature_dict=feature_dict)

In [14]:
def load_model(path, model, cuda='cuda:1'):
    # load pretrained model and update weights
    pretrained_dict = torch.load(path)
    model_dict = model.state_dict()
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v.to(cuda) for k, v in pretrained_dict.items()}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    # 3. load the new state dict
    model.load_state_dict(model_dict)
    return model

model_GPAbs = load_model('/home/shared/yikuan/HF/model/PureICD/GPabs_diag_med_MLM_20_40_20_best.bin', model_GPAbs, cuda_gpAbs)
model_GPAbs = model_GPAbs.to(cuda_gpAbs)

model_GPAdditive = load_model('/home/shared/yikuan/HF/model/PureICD/GPAdditive_diag_med_MLM_test_best.bin', model_GPAdditive, cuda_gpAdditive)
model_GPAdditive = model_GPAdditive.to(cuda_gpAdditive)

model_BERT = load_model('/home/shared/yikuan/HF/model/PureICD/ModelA_diag_med_MLM_best.bin', model_BERT, cuda_bert)
model_BERT = model_BERT.to(cuda_bert)

model_BNNGP = load_model('/home/shared/yikuan/Model/BayesianGPBERT/BayesianBERTGP_diag_med_5_best.bin', model_BNNGP, cuda_bnngp_0)
model_BNNGP = model_BNNGP.to(cuda_bnngp_0)
model_BNNGP.allocateGPU()

model_BNN = load_model('/home/shared/yikuan/Model/BayesianGPBERT/BayesianBERT_diag_med_150_posterior_best.bin', model_BNN, cuda_bnn)
model_BNN = model_BNN.to(cuda_bnn)

model_BNNClassifier = load_model('/home/shared/yikuan/HF/model/PureICD/BNN_1_diag_med_MLM_best.bin', model_BNNClassifier, cuda_bnnclassifier)
model_BNNClassifier = model_BNNClassifier.to(cuda_bnnclassifier)

model_EmbClassifier = load_model('/home/shared/yikuan/Model/BayesianGPBERT/BayesianBERTClassifier_diag_med_150_posterior_best.bin', model_EmbClassifier, cuda_embclassifier)
model_EmbClassifier = model_EmbClassifier.to(cuda_embclassifier)

In [15]:
# def precision(logits, label):
#     sig = nn.Sigmoid()
#     output=sig(logits)
#     label, output=label.cpu(), output.detach().cpu()
#     tempprc= average_precision_score(label.numpy(),output.numpy())
#     return tempprc, output, label

def precision_GP(logits, label):
#     sig = nn.Sigmoid()
#     output=sig(logits)
    tempprc= average_precision_score(label.numpy(),logits.numpy())
    return tempprc, logits,label

def precision_BERT(logits, label):
    sig = nn.Sigmoid()
    output=sig(logits)
    tempprc= average_precision_score(label.numpy(),output.numpy())
    return tempprc, output, label

def roc_score_GP(logits, label):
    score= roc_auc_score(label.numpy(),logits.numpy())
    return score, logits,label

def roc_score_BERT(logits, label):
    sig = nn.Sigmoid()
    output=sig(logits)
    score= roc_auc_score(label.numpy(),output.numpy())
    return score, output, label

In [16]:
def evaluation_GP(testload, model, likelihood, cuda, n_sample=10):
    model.eval()
    y = []
    y_label = []
    variance_list = []
    predictive_list = []
    epis_list = []
    alea_list = []
    age_list = []
    patid_list = []
    
    loss_temp = 0

#     model.eval()
    likelihood.eval()
    with torch.no_grad():
        for step, batch in enumerate(testload):
            age_ids, input_ids, posi_ids, segment_ids, attMask, targets, patid, last_age = batch

            age_ids = age_ids.to(cuda)
            input_ids = input_ids.to(cuda)
            posi_ids = posi_ids.to(cuda)
            segment_ids = segment_ids.to(cuda)
            attMask = attMask.to(cuda)
            targets = targets.view(-1).to(cuda)
            last_age = last_age.view(-1)
            
            output_list = []
            for _ in range(n_sample):
                output = model(input_ids, age_ids, segment_ids, posi_ids, attention_mask=attMask, labels=targets)
                output_list.append(output.sample())
                
            logits = torch.sigmoid(torch.stack(output_list, dim=1)).cpu()
#             alea, epis, predictive, variance = uncertain_cal(torch.sigmoid(torch.stack(output_list, dim=1)), dim=1)
            targets = targets.cpu()
            
#             alea_list.append(alea.cpu())
#             epis_list.append(epis.cpu())
#             variance_list.append(variance.cpu())
#             predictive_list.append(predictive.cpu())
            
            y_label.append(targets)
            y.append(logits)
#             variance_list.append(variance)
            patid_list.append(patid.view(-1))
            age_list.append(last_age)
            
            if step % 500 == 0:
                print('GP: {}'.format(step))

        y_label = torch.cat(y_label, dim=0).view(-1)
        y = torch.cat(y, dim=0)
#         variance_list = torch.cat(variance_list, dim=0).view(-1)
        patid_list = torch.cat(patid_list, dim=0).view(-1)
        age_list = torch.cat(age_list, dim=0).view(-1)
#         alea_list = torch.cat(alea_list, dim=0).view(-1)
#         epis_list = torch.cat(epis_list,dim=0).view(-1)
#         predictive_list = torch.cat(predictive_list, dim=0).view(-1)

#         tempprc, output, label = precision_GP(y.view(-1), y_label.view(-1))
#         score, _, _ = roc_score_GP(y.view(-1), y_label.view(-1))
#         return tempprc, score, y, y_label, variance_list, patid_list, age_list, epis_list, alea_list, predictive_list
        return y_label, y, patid_list, age_list

def evaluation_BNNGP(testload, model, likelihood, cuda1, cuda2, n_sample=10):
#     model.eval()
    model.eval()
    likelihood.eval()

    y = []
    y_label = []
    variance_list = []
    epis_list = []
    alea_list = []
    age_list = []
    patid_list = []
    predictive_list = []
    
    loss_temp = 0

#     model.eval()
#     likelihood.eval()
    with torch.no_grad():
        for step, batch in enumerate(testload):
            age_ids, input_ids, posi_ids, segment_ids, attMask, targets, patid, last_age = batch

            age_ids = age_ids.to(cuda1)
            input_ids = input_ids.to(cuda1)
            posi_ids = posi_ids.to(cuda1)
            segment_ids = segment_ids.to(cuda1)
            attMask = attMask.to(cuda1)
            targets = targets.view(-1).to(cuda1)
            last_age = last_age.view(-1)
#             output = likelihood(
#                 model(input_ids, age_ids, segment_ids, posi_ids, attention_mask=attMask, labels=targets)[0])
            output_list = []
            for _ in range(n_sample):
                output, kl = model(input_ids, age_ids, segment_ids, posi_ids, attention_mask=attMask, labels=targets)
                output_list.append(output.sample())
                
            logits = torch.sigmoid(torch.stack(output_list, dim=1)).cpu()
#             alea, epis, predictive, variance = uncertain_cal(torch.sigmoid(torch.stack(output_list, dim=1)), dim=1)
            targets = targets.cpu()
            
#             alea_list.append(alea.cpu())
#             epis_list.append(epis.cpu())
#             variance_list.append(variance.cpu())
#             predictive_list.append(predictive.cpu())

            y_label.append(targets)
            y.append(logits)
#             variance_list.append(variance)
            patid_list.append(patid.view(-1))
            age_list.append(last_age)
            
            if step % 500 == 0:
                print('BNNGP: {}'.format(step))    
        
        y_label = torch.cat(y_label, dim=0).view(-1)
        y = torch.cat(y, dim=0)
#         variance_list = torch.cat(variance_list, dim=0).view(-1)
        patid_list = torch.cat(patid_list, dim=0).view(-1)
        age_list = torch.cat(age_list, dim=0).view(-1)
#         alea_list = torch.cat(alea_list, dim=0).view(-1)
#         epis_list = torch.cat(epis_list,dim=0).view(-1)
#         predictive_list = torch.cat(predictive_list, dim=0).view(-1)

#         tempprc, output, label = precision_GP(y.view(-1), y_label.view(-1))
#         score, _, _ = roc_score_GP(y.view(-1), y_label.view(-1))
#         return tempprc, score, y, y_label, variance_list, patid_list, age_list, epis_list, alea_list, predictive_list
        return y_label, y, patid_list, age_list
        
def evaluation_BNN(testload, model, cuda, n_sample=10):
    model.eval()
    y = []
    y_label = []
    variance_list = []
    epis_list = []
    alea_list = []
    age_list = []
    patid_list = []
    predictive_list = []
    
    loss_temp = 0
    
    with torch.no_grad():
        for step, batch in enumerate(testload):
            age_ids, input_ids, posi_ids, segment_ids, attMask, targets, patid, last_age = batch

            age_ids = age_ids.to(cuda)
            input_ids = input_ids.to(cuda)
            posi_ids = posi_ids.to(cuda)
            segment_ids = segment_ids.to(cuda)
            attMask = attMask.to(cuda)
            targets = targets.view(-1).to(cuda)
            last_age = last_age.view(-1)
            
            logits_prob = []
            for i in range(n_sample):
                logits =model(input_ids, age_ids, segment_ids, posi_ids, attention_mask=attMask, labels=None)
                logits_prob.append(logits)
            
            logits = torch.sigmoid(torch.stack(logits_prob, dim=1)).cpu()
#             alea, epis, predictive,variance = uncertain_cal(torch.sigmoid(torch.stack(logits_prob, dim=1)), dim=1)
            targets = targets.cpu()
            
#             alea_list.append(alea.cpu())
#             epis_list.append(epis.cpu())
#             variance_list.append(variance.cpu())
#             predictive_list.append(predictive.cpu())
                
            y_label.append(targets)
            y.append(logits)
#             variance_list.append(variance)
            patid_list.append(patid.view(-1))
            age_list.append(last_age)
            
            if step % 500 == 0:
                print('BNN: {}'.format(step))
                
        y_label = torch.cat(y_label, dim=0).view(-1)
        y = torch.cat(y, dim=0)
#         variance_list = torch.cat(variance_list, dim=0).view(-1)
        patid_list = torch.cat(patid_list, dim=0).view(-1)
        age_list = torch.cat(age_list, dim=0).view(-1)
#         alea_list = torch.cat(alea_list, dim=0).view(-1)
#         epis_list = torch.cat(epis_list,dim=0).view(-1)
#         predictive_list = torch.cat(predictive_list, dim=0).view(-1)

#         tempprc, output, label = precision_GP(y.view(-1), y_label.view(-1))
#         score, _, _ = roc_score_GP(y.view(-1), y_label.view(-1))

#         return tempprc, score, y, y_label, variance_list, patid_list, age_list, epis_list, alea_list, predictive_list
        return y_label, y, patid_list, age_list

    
    
def evaluation_BERT(testload, model, cuda):
    model.eval()
    y = []
    y_label = []
    age_list = []
    patid_list = []
    
    loss_temp = 0
    with torch.no_grad():
        for step, batch in enumerate(testload):
            age_ids, input_ids, posi_ids, segment_ids, attMask, targets, patid, last_age = batch

            age_ids = age_ids.to(cuda)
            input_ids = input_ids.to(cuda)
            posi_ids = posi_ids.to(cuda)
            segment_ids = segment_ids.to(cuda)
            attMask = attMask.to(cuda)
            targets = targets.view(-1).to(cuda)
            last_age = last_age.view(-1)
            patid_list.append(patid.view(-1))
            age_list.append(last_age)

            logits =model(input_ids, age_ids, segment_ids, posi_ids, attention_mask=attMask, labels=None)
            targets = targets.cpu()

            y_label.append(targets)
            y.append(logits.cpu())
            
            if step % 500 == 0:
                print('BERT: {}'.format(step))

        y_label = torch.cat(y_label, dim=0).view(-1)
        y = torch.cat(y, dim=0).view(-1)
        patid_list = torch.cat(patid_list, dim=0).view(-1)
        age_list = torch.cat(age_list, dim=0).view(-1)

#         tempprc, output, label = precision_BERT(y.view(-1), y_label.view(-1))
#         score, _, _ = roc_score_BERT(y.view(-1), y_label.view(-1))
#         return tempprc, score, patid_list, age_list, output, y_label
        return y_label, y, patid_list, age_list

In [17]:
recorder = H5Recorder('/home/shared/yikuan/ACM/data/HeartFailure/bertWhitenedGP_60.h5')
recorder.open(read=False)
label, y, patid, age = evaluation_GP(testData, model_GPAbs, likelihoodAbs, cuda_gpAbs, n_sample=60)
recorder.write('label', label.numpy())
recorder.write('prob', y.numpy())
recorder.write('age', age.numpy())
recorder.write('patid', patid.numpy())
recorder.close()

GP: 0
GP: 500
GP: 1000
GP: 1500
GP: 2000
GP: 2500
GP: 3000
GP: 3500


In [18]:
recorder = H5Recorder('/home/shared/yikuan/ACM/data/HeartFailure/bertKISSGP_60.h5')
recorder.open(read=False)
label, y, patid, age = evaluation_GP(testData, model_GPAdditive, likelihoodAdditive, cuda_gpAdditive, n_sample=60)
recorder.write('label', label.numpy())
recorder.write('prob', y.numpy())
recorder.write('age', age.numpy())
recorder.write('patid', patid.numpy())
recorder.close()

GP: 0
GP: 500
GP: 1000
GP: 1500
GP: 2000
GP: 2500
GP: 3000
GP: 3500


In [18]:
# recorder = H5Recorder('/home/shared/yikuan/ACM/data/HeartFailure/bert.h5')
# recorder.open(read=False)
# label, y, patid, age = evaluation_BERT(testData, model_BERT, cuda_bert)
# recorder.write('label', label.numpy())
# recorder.write('prob', y.numpy())
# recorder.write('age', age.numpy())
# recorder.write('patid', patid.numpy())
# recorder.close()

BERT: 0
BERT: 500
BERT: 1000
BERT: 1500
BERT: 2000
BERT: 2500
BERT: 3000
BERT: 3500


In [19]:
recorder = H5Recorder('/home/shared/yikuan/ACM/data/HeartFailure/bertBayesianEmbedding_60.h5')
recorder.open(read=False)
label, y, patid, age = evaluation_BNN(testData, model_BNN, cuda_bnn, n_sample=60)
recorder.write('label', label.numpy())
recorder.write('prob', y.numpy())
recorder.write('age', age.numpy())
recorder.write('patid', patid.numpy())
recorder.close()

BNN: 0
BNN: 500
BNN: 1000
BNN: 1500
BNN: 2000
BNN: 2500
BNN: 3000
BNN: 3500


In [20]:
recorder = H5Recorder('/home/shared/yikuan/ACM/data/HeartFailure/bertBayesianOutput_60.h5')
recorder.open(read=False)
label, y, patid, age = evaluation_BNN(testData, model_BNNClassifier, cuda_bnnclassifier, n_sample=60)
recorder.write('label', label.numpy())
recorder.write('prob', y.numpy())
recorder.write('age', age.numpy())
recorder.write('patid', patid.numpy())
recorder.close()

BNN: 0
BNN: 500
BNN: 1000
BNN: 1500
BNN: 2000
BNN: 2500
BNN: 3000
BNN: 3500


In [21]:
recorder = H5Recorder('/home/shared/yikuan/ACM/data/HeartFailure/bertBayesianEmbeddingOutput_60.h5')
recorder.open(read=False)
label, y, patid, age = evaluation_BNN(testData, model_EmbClassifier, cuda_embclassifier, n_sample=60)
recorder.write('label', label.numpy())
recorder.write('prob', y.numpy())
recorder.write('age', age.numpy())
recorder.write('patid', patid.numpy())
recorder.close()

BNN: 0
BNN: 500
BNN: 1000
BNN: 1500
BNN: 2000
BNN: 2500
BNN: 3000
BNN: 3500


In [22]:
recorder = H5Recorder('/home/shared/yikuan/ACM/data/HeartFailure/bertDBKL_60.h5')
recorder.open(read=False)
label, y, patid, age = evaluation_BNNGP(testData, model_BNNGP, likelihoodBNNGP, cuda_bnngp_0, cuda_bnngp_1, n_sample=60)
recorder.write('label', label.numpy())
recorder.write('prob', y.numpy())
recorder.write('age', age.numpy())
recorder.write('patid', patid.numpy())
recorder.close()

BNNGP: 0
BNNGP: 500
BNNGP: 1000
BNNGP: 1500
BNNGP: 2000
BNNGP: 2500
BNNGP: 3000
BNNGP: 3500


In [23]:
# # print('number of iteration:', i)
# auc_GPAbs, roc_GPAbs, y_GPAbs, label_GPAbs, var_GPAbs, patid_GPAbs, age_GPAbs,epis_GPAbs, alea_GPAbs, predictive_GPAbs = evaluation_GP(testData, model_GPAbs, likelihoodAbs, cuda_gpAbs, n_sample=50)
# print('GPAbs precision: {}, ROC: {}'.format(auc_GPAbs, roc_GPAbs))
# auc_GPAdditive, roc_GPAdditive, y_GPAdditive, label_GPAdditive, var_GPAdditive, patid_GPAdditive, age_GPAdditive, epis_GPAdditive, alea_GPAdditive, predictive_GPAdditive = evaluation_GP(testData, model_GPAdditive, likelihoodAdditive, cuda_gpAdditive, n_sample=50)
# print('GPAdditive precision: {}, ROC: {}'.format(auc_GPAdditive, roc_GPAdditive))
# auc_BERT, roc_BERT, patid_BERT, age_BERT, y_Bert, label_Bert = evaluation_BERT(testData, model_BERT, cuda_bert)
# print('BERT precision: {}, ROC: {}'.format(auc_BERT, roc_BERT))
# auc_BNNEmb, roc_BNNEmb, y_BNNEmb, label_BNNEmb, var_BNNEmb, patid_BNNEmb, age_BNNEmb, epis_BNNEmb, alea_BNNEmb, predictive_BNNEmb = evaluation_BNN(testData, model_BNN, cuda_bnn, n_sample=50)
# print('BNNEmb precision: {}, ROC: {}'.format(auc_BNNEmb, roc_BNNEmb))
# auc_BNNClassifier, roc_BNNClassifier, y_BNNClassifier, label_BNNClassifier, var_BNNClassifier, patid_BNNClassifier, age_BNNClassifier, epis_BNNClassifier, alea_BNNClassifier, predictive_BNNClassifier = evaluation_BNN(testData, model_BNNClassifier, cuda_bnnclassifier, n_sample=50)
# print('BNNClassifier precision: {}, ROC: {}'.format(auc_BNNClassifier, roc_BNNClassifier))
# auc_EmbClassifier, roc_EmbClassifier, y_EmbClassifier, label_EmbClassifier, var_EmbClassifier, patid_EmbClassifier, age_EmbClassifier, epis_EmbClassifier, alea_EmbClassifier, predictive_EmbClassifier = evaluation_BNN(testData, model_EmbClassifier, cuda_embclassifier, n_sample=50)
# print('EmbClassifier precision: {}, ROC: {}'.format(auc_EmbClassifier, roc_EmbClassifier))


# df_abs =pd.DataFrame({
#     'label':label_GPAbs.numpy(),
#     'p_abs': y_GPAbs.numpy(),
#     'var_abs': var_GPAbs.numpy(),
#     'epis_abs': epis_GPAbs.numpy(),
#     'alea_abs': alea_GPAbs.numpy(),
#     'pred_abs': predictive_GPAbs.numpy(),
#     'patid': patid_GPAbs.numpy(),
#     'age': age_GPAbs.numpy()
# })

# df_additive = pd.DataFrame({
# #     'label':label_GPAdditive.numpy(),
#     'p_additive': y_GPAdditive.numpy(),
#     'var_additive': var_GPAdditive.numpy(),
#     'epis_additive': epis_GPAdditive.numpy(),
#     'alea_additive': alea_GPAdditive.numpy(),
#     'pred_additive': predictive_GPAdditive.numpy(),
#     'patid': patid_GPAdditive.numpy(),
# #     'age': age_GPAdditive.numpy()
# })

# df_bert = pd.DataFrame({
# #     'label':label_BERT.numpy(),
#     'p_bert': y_Bert.numpy(),
# #     'var_additive': var_GPAdditive.numpy(),
#     'patid': patid_BERT.numpy(),
# #     'age': age_BERT.numpy()
# })

# df_bnnemb = pd.DataFrame({
# #     'label':label_BERT.numpy(),
#     'p_bnnemb': y_BNNEmb.numpy(),
#     'var_bnnemb': var_BNNEmb.numpy(),
#     'epis_bnnemb': epis_BNNEmb.numpy(),
#     'alea_bnnemb': alea_BNNEmb.numpy(),
#     'pred_bnnemb': predictive_BNNEmb.numpy(),
#     'patid': patid_BNNEmb.numpy(),
# #     'age': age_BERT.numpy()
# })

# df_bnnembgp = pd.DataFrame({
# #     'label':label_BERT.numpy(),
#     'p_bnngp': y_BNNEmbGP.numpy(),
#     'var_bnngp': var_BNNEmbGP.numpy(),
#     'epis_bnngp': epis_BNNEmbGP.numpy(),
#     'alea_bnngp': alea_BNNEmbGP.numpy(),
#     'pred_bnngp': predictive_BNNEmbGP.numpy(),
#     'patid': patid_BNNEmbGP.numpy(),
# #     'age': age_BERT.numpy()
# })

# df_bnnclassifier = pd.DataFrame({
# #     'label':label_BERT.numpy(),
#     'p_bnnclassifier': y_BNNClassifier.numpy(),
#     'var_bnnclassifier': var_BNNClassifier.numpy(),
#     'epis_bnnclassifier': epis_BNNClassifier.numpy(),
#     'alea_bnnclassifier': alea_BNNClassifier.numpy(),
#     'pred_bnnclassifier': predictive_BNNClassifier.numpy(),
#     'patid': patid_BNNClassifier.numpy(),
# #     'age': age_BERT.numpy()
# })

# df_embclassifier = pd.DataFrame({
# #     'label':label_BERT.numpy(),
#     'p_embclassifier': y_EmbClassifier.numpy(),
#     'var_embclassifier': var_EmbClassifier.numpy(),
#     'epis_embclassifier': epis_EmbClassifier.numpy(),
#     'alea_embclassifier': alea_EmbClassifier.numpy(),
#     'pred_embclassifier': predictive_EmbClassifier.numpy(),
#     'patid': patid_EmbClassifier.numpy(),
# #     'age': age_BERT.numpy()
# })

# df = df_abs.merge(df_additive, on='patid').merge(df_bert, on='patid').merge(df_bnnemb, on='patid').merge(df_bnnembgp, on='patid').merge(df_bnnclassifier,on='patid').merge(df_embclassifier,on='patid')
# df.to_parquet('/home/shared/yikuan/ACM/data/Diabetes/summary.parquet')

In [24]:
# auc_BNNEmbGP, roc_BNNEmbGP, y_BNNEmbGP, label_BNNEmbGP, var_BNNEmbGP, patid_BNNEmbGP, age_BNNEmbGP,epis_BNNEmbGP, alea_BNNEmbGP, predictive_BNNEmbGP = evaluation_BNNGP(testData, model_BNNGP, likelihoodBNNGP, cuda_bnngp_0, cuda_bnngp_1, n_sample=30)
# print('BNNGP precision: {}, ROC: {}'.format(auc_BNNEmbGP, roc_BNNEmbGP))

# df_bnnembgp = pd.DataFrame({
#     'label':label_BNNEmbGP.numpy(),
#     'p_bnngp': y_BNNEmbGP.numpy(),
#     'var_bnngp': var_BNNEmbGP.numpy(),
#     'epis_bnngp': epis_BNNEmbGP.numpy(),
#     'alea_bnngp': alea_BNNEmbGP.numpy(),
#     'pred_bnngp': predictive_BNNEmbGP.numpy(),
#     'patid': patid_BNNEmbGP.numpy(),
#     'age': age_BNNEmbGP.numpy()
# })

# df_bnnembgp.to_parquet('/home/shared/yikuan/ACM/data/Diabetes/summary_DBKL.parquet')